In [1]:
import os
import logging
import sqlite3
from datetime import datetime

import pandas as pd
import numpy as np

# ==============================
#  CONFIG
# ==============================
BASE_PATH = r"E:\work\DEPI\graduation promax"

RAW_DB = fr"{BASE_PATH}\data\raw\ivf_database_updated.db"
STAR_DB = fr"{BASE_PATH}\data\warehouse\ivf_star_schema.db"
SCHEMA_SQL = fr"{BASE_PATH}\src\etl\create_star_schema.sql"
LOG_FILE = fr"{BASE_PATH}\src\etl\logs\etl_log_ivf.txt"

os.makedirs(os.path.dirname(LOG_FILE), exist_ok=True)
os.makedirs(os.path.dirname(STAR_DB), exist_ok=True)

# ==============================
#  LOGGING
# ==============================
logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s"
)
logging.info("===== ETL STARTED =====")


# ==============================
#  UTILS
# ==============================
def run_schema_sql():
    """
    FULL REFRESH للـ star schema:
    - Drop لكل الجداول
    - Create من ملف create_star_schema.sql
    """
    logging.info("Creating star schema from SQL file...")
    conn = sqlite3.connect(STAR_DB)
    with open(SCHEMA_SQL, "r", encoding="utf-8") as f:
        sql_script = f.read()
    conn.executescript(sql_script)
    conn.commit()
    conn.close()
    logging.info("Star schema created successfully.")


def load_raw_df() -> pd.DataFrame:
    """تحميل جدول ivf_patients من RAW DB"""
    if not os.path.exists(RAW_DB):
        raise FileNotFoundError(f"RAW DB not found at: {RAW_DB}")

    conn = sqlite3.connect(RAW_DB)
    tables = pd.read_sql(
        "SELECT name FROM sqlite_master WHERE type='table';", conn
    )["name"].tolist()

    if "ivf_patients" not in tables:
        conn.close()
        raise RuntimeError("Table 'ivf_patients' not found in RAW DB.")

    df = pd.read_sql("SELECT * FROM ivf_patients;", conn)
    conn.close()
    logging.info(f"Loaded {len(df)} rows from raw table.")
    return df


# ==============================
#  CLEANING
# ==============================
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    logging.info("Cleaning started...")

    # Standardize column names
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
    )

    # Drop exact duplicates
    before = len(df)
    df = df.drop_duplicates()
    logging.info(f"Dropped {before - len(df)} duplicate rows.")

    # Strip strings
    str_cols = df.select_dtypes(include="object").columns
    for c in str_cols:
        df[c] = df[c].astype(str).str.strip()

    # Numeric missing + winsorization
    num_cols = df.select_dtypes(include=["float", "int"]).columns
    for c in num_cols:
        if df[c].isna().any():
            med = df[c].median()
            df[c] = df[c].fillna(med)
            logging.info(f"Filled NaN in {c} with median={med}.")
        q1 = df[c].quantile(0.01)
        q99 = df[c].quantile(0.99)
        df[c] = df[c].clip(q1, q99)

    # String missing
    for c in str_cols:
        if df[c].isna().any():
            df[c] = df[c].fillna("Unknown")

    logging.info("Cleaning finished.")
    return df


# ==============================
#  PLACEHOLDER & ID GENERATION
# ==============================
def apply_placeholder_and_ids(df: pd.DataFrame) -> pd.DataFrame:
    logging.info("Applying placeholder rules & generating IDs...")

    # -------- Basic clinical fields --------
    if "risk_level" not in df.columns:
        df["risk_level"] = "Unknown"

    if "good_embryos" not in df.columns:
        df["good_embryos"] = 0

    if "embryos_transferred" not in df.columns:
        df["embryos_transferred"] = 0

    if "et_date" not in df.columns:
        df["et_date"] = datetime.today().strftime("%Y-%m-%d")

    # Suggested Waiting Period by risk
    df["suggested_waiting_period_days"] = df["risk_level"].map({
        "High": 90,
        "Medium": 45,
        "Low": 30
    }).fillna(30)

    # Class A Rate
    denom = df["embryos_transferred"].replace(0, np.nan)
    df["class_a_rate"] = (df["good_embryos"] / denom).fillna(0)

    # -------- Doctor --------
    if "doctor_id" not in df.columns:
        if "doctor_name" in df.columns:
            df["doctor_id"] = (
                "dr_" +
                df["doctor_name"].astype(str)
                .str.lower().str.replace(" ", "_")
            )
        else:
            df["doctor_id"] = "dr_unknown"
    df["doctor_id"] = df["doctor_id"].fillna("dr_unknown")

    if "doctor_recommendation" not in df.columns:
        df["doctor_recommendation"] = "Unknown"

    # -------- Protocol --------
    if "protocol_type" not in df.columns:
        df["protocol_type"] = "Unknown"
    if "stimulation_days" not in df.columns:
        df["stimulation_days"] = 0
    if "total_fsh_dose" not in df.columns:
        df["total_fsh_dose"] = 0
    if "trigger_type" not in df.columns:
        df["trigger_type"] = "Unknown"
    if "recommended_protocol" not in df.columns:
        df["recommended_protocol"] = df["doctor_recommendation"]

    # protocol_id deterministic based on features
    df["protocol_id"] = (
        "prot_"
        + df["protocol_type"].astype(str).str.lower().str.replace(" ", "_")
        + "_d" + df["stimulation_days"].astype(str)
        + "_dose" + df["total_fsh_dose"].astype(str)
        + "_trg_" + df["trigger_type"].astype(str).str.lower().str.replace(" ", "_")
    )

    # -------- Outcome --------
    if "response_type" not in df.columns:
        df["response_type"] = "Unknown"
    if "failure_reason" not in df.columns:
        df["failure_reason"] = "Unknown"

    df["outcome_id"] = (
        "out_"
        + df["risk_level"].astype(str).str.lower().str.replace(" ", "_")
        + "_"
        + df["response_type"].astype(str).str.lower().str.replace(" ", "_")
    )

    # -------- Embryo --------
    if "fresh_et_stage" not in df.columns:
        df["fresh_et_stage"] = "Unknown"
    if "grading" not in df.columns:
        df["grading"] = "Unknown"

    if "embryo_id" not in df.columns:
        df["embryo_id"] = (
            "emb_"
            + df["fresh_et_stage"].astype(str).str.lower().str.replace(" ", "_")
            + "_"
            + df["grading"].astype(str).str.lower().str.replace(" ", "_")
            + "_"
            + df.index.astype(str)
        )

    # -------- M2 & injected_m2 --------
    if "m2_count" not in df.columns:
        df["m2_count"] = 0
    else:
        df["m2_count"] = df["m2_count"].fillna(0).astype(int)

    if "injected_m2" not in df.columns:
        df["injected_m2"] = df["m2_count"]
    else:
        df["injected_m2"] = df["injected_m2"].fillna(df["m2_count"]).astype(int)

    # -------- Outcome related --------
    if "clinical_pregnancy" not in df.columns:
        df["clinical_pregnancy"] = 0
    if "live_birth" not in df.columns:
        df["live_birth"] = 0
    if "success_probability_score" not in df.columns:
        df["success_probability_score"] = 0.0

    # Transfer time id (date text)
    df["transfer_time_id"] = pd.to_datetime(
        df.get("transfer_time_id", df["et_date"]),
        errors="coerce"
    ).dt.strftime("%Y-%m-%d")

    logging.info("Placeholder rules & IDs generated.")
    return df


# ==============================
#  LOAD DIMENSIONS
# ==============================
def load_dimensions_full_refresh(df: pd.DataFrame, conn: sqlite3.Connection):
    """
    FULL REFRESH MODE — reload all tables, no duplicate logic.
    Use ONLY when doing full rebuild (after DROP TABLE).
    """

    logging.info("Loading dimensions (FULL REFRESH MODE)...")

    tables = {
        "dim_female": [
            "female_id", "female_age", "female_bmi", "amh_level", "fsh_level", "afc"
        ],
        "dim_male": [
            "male_id", "male_age", "male_factor",
            "semen_count_mill_per_ml", "motility_percent", "morphology_percent"
        ],
        "dim_protocol": [
            "protocol_id", "protocol_type", "stimulation_days",
            "total_fsh_dose", "trigger_type", "recommended_protocol"
        ],
        "dim_doctor": [
            "doctor_id", "doctor_recommendation"
        ],
        "dim_outcome": [
            "outcome_id", "risk_level", "response_type",
            "suggested_waiting_period_days", "failure_reason"
        ],
        "dim_embryo": [
            "embryo_id", "fresh_et_stage", "grading", "class_a_rate"
        ]
    }

    for table_name, cols in tables.items():
        if not set(cols).issubset(df.columns):
            logging.warning(f"{table_name} skipped — missing cols: {set(cols) - set(df.columns)}")
            continue

        dim = df[cols].drop_duplicates()
        dim.to_sql(table_name, conn, if_exists="replace", index=False)
        logging.info(f"{table_name}: inserted {len(dim)} rows.")



# ==============================
#  LOAD FACTS
# ==============================
def load_fact_ivf_cycle(df: pd.DataFrame, conn: sqlite3.Connection) -> None:
    logging.info("Loading fact_ivf_cycle...")

    required = [
        "case_id", "female_id", "male_id", "protocol_id", "doctor_id",
        "outcome_id", "et_date", "e2_on_trigger", "endometrium_thickness",
        "follicles_18mm", "retrieved_oocytes", "m2_count", "gv_count",
        "injected_m2", "fertilized_oocytes", "fertilization_rate",
        "cleavage_d3", "blastocyst_d5", "good_embryos"
    ]
    missing = [c for c in required if c not in df.columns]
    if missing:
        logging.error(f"Missing columns for fact_ivf_cycle: {missing}")
        return

    # Map dates to time_id
    time_df = pd.read_sql("SELECT time_id, full_date FROM dim_time;", conn)
    if time_df.empty:
        logging.error("dim_time is empty – cannot link fact_ivf_cycle to time dimension.")
        return

    date_to_id = dict(zip(time_df["full_date"], time_df["time_id"]))

    tmp = df.copy()
    tmp["et_date_clean"] = pd.to_datetime(tmp["et_date"], errors="coerce").dt.strftime("%Y-%m-%d")
    tmp["cycle_start_time_id"] = tmp["et_date_clean"].map(date_to_id)

    fact = tmp[[
        "case_id", "female_id", "male_id", "protocol_id", "doctor_id",
        "outcome_id", "cycle_start_time_id", "e2_on_trigger",
        "endometrium_thickness", "follicles_18mm", "retrieved_oocytes",
        "m2_count", "gv_count", "injected_m2", "fertilized_oocytes",
        "fertilization_rate", "cleavage_d3", "blastocyst_d5", "good_embryos"
    ]].drop_duplicates(subset=["case_id"])

    fact.to_sql("fact_ivf_cycle", conn, if_exists="append", index=False)
    logging.info(f"fact_ivf_cycle: {len(fact)} rows.")


def load_fact_transfer(df: pd.DataFrame, conn: sqlite3.Connection) -> None:
    logging.info("Loading fact_transfer...")

    required = [
        "case_id", "doctor_id", "transfer_time_id", "embryos_transferred",
        "pregnancy_test_result", "clinical_pregnancy", "live_birth",
        "outcome_id", "success_probability_score"
    ]
    missing = [c for c in required if c not in df.columns]
    if missing:
        logging.warning(f"Missing columns for fact_transfer: {missing}")
        return

    time_df = pd.read_sql("SELECT time_id, full_date FROM dim_time;", conn)
    if time_df.empty:
        logging.error("dim_time is empty – cannot link fact_transfer to time dimension.")
        return

    date_to_id = dict(zip(time_df["full_date"], time_df["time_id"]))

    tmp = df.copy()
    tmp["transfer_date_clean"] = pd.to_datetime(tmp["transfer_time_id"], errors="coerce").dt.strftime("%Y-%m-%d")
    tmp["transfer_time_fk"] = tmp["transfer_date_clean"].map(date_to_id)

    fact_t = tmp[[
        "case_id", "doctor_id", "transfer_time_fk", "embryos_transferred",
        "pregnancy_test_result", "clinical_pregnancy", "live_birth",
        "outcome_id", "success_probability_score"
    ]].drop_duplicates(subset=["case_id"])

    fact_t.to_sql("fact_transfer", conn, if_exists="append", index=False)
    logging.info(f"fact_transfer: {len(fact_t)} rows.")


# (اختياري) fact_transfer_embryo – هنسيبه فاضي لحد ما يبقى عندنا rows على مستوى embryo


def load_dim_time(df: pd.DataFrame, conn: sqlite3.Connection):
    """
    Create dim_time table WITHOUT dropping existing structure.
    """
    logging.info("Building dim_time...")

    if "et_date" not in df.columns:
        logging.error("No 'et_date' column — cannot build dim_time.")
        return

    # Extract unique dates
    dates = pd.to_datetime(df["et_date"], errors="coerce").dropna().drop_duplicates()
    time_dim = pd.DataFrame({
        "full_date": dates.dt.strftime("%Y-%m-%d"),
        "day": dates.dt.day,
        "month": dates.dt.month,
        "month_name": dates.dt.month_name(),
        "quarter": dates.dt.quarter,
        "year": dates.dt.year,
        "week": dates.dt.isocalendar().week.astype(int)
    })

    # Append — NOT Replace!
    time_dim.to_sql("dim_time", conn, if_exists="append", index=False)
    logging.info(f"dim_time inserted {len(time_dim)} rows.")

    # Make sure time_id exists
    chk = pd.read_sql("PRAGMA table_info(dim_time);", conn)
    if "time_id" not in chk["name"].values:
        logging.warning("time_id missing — regenerating...")
        conn.executescript("""
            CREATE TABLE dim_time_temp AS
            SELECT 
                ROW_NUMBER() OVER () AS time_id,
                full_date, day, month, month_name, quarter, year, week
            FROM dim_time;

            DROP TABLE dim_time;
            ALTER TABLE dim_time_temp RENAME TO dim_time;
        """)
        conn.commit()

    logging.info("dim_time ready.")


# ==============================
#  MAIN
# ==============================
def run_full_etl():
    try:
        run_schema_sql()                # 1) rebuild schema
        df = load_raw_df()              # 2) raw → df
        df = clean_data(df)             # 3) cleaning
        df = apply_placeholder_and_ids(df)  # 4) placeholder + IDs

        conn = sqlite3.connect(STAR_DB)

        load_dimensions_full_refresh(df, conn) # 5) dimensions
        load_dim_time(df, conn)       
        load_fact_ivf_cycle(df, conn)   # 6) fact_ivf_cycle
        load_fact_transfer(df, conn)    # 7) fact_transfer

        conn.close()
        logging.info("ETL COMPLETED SUCCESSFULLY.")
        print("ETL COMPLETED SUCCESSFULLY. Check warehouse DB and log file.")

    except Exception as e:
        logging.exception(f"ETL FAILED: {e}")
        print("ETL FAILED. Check log file for details:", e)


if __name__ == "__main__":
    run_full_etl()


ETL COMPLETED SUCCESSFULLY. Check warehouse DB and log file.


In [3]:
import sqlite3
import pandas as pd

db_path = STAR_DB
conn = sqlite3.connect(db_path)

# Show table structure (columns + constraints)
table_info = pd.read_sql("PRAGMA table_info(dim_protocol);", conn)
print(table_info)

# Show existing values in dim_protocol
data = pd.read_sql("SELECT * FROM dim_protocol;", conn)
print(data.head())

conn.close()


   cid                  name     type  notnull dflt_value  pk
0    0           protocol_id     TEXT        0       None   0
1    1         protocol_type     TEXT        0       None   0
2    2      stimulation_days  INTEGER        0       None   0
3    3        total_fsh_dose     REAL        0       None   0
4    4          trigger_type     TEXT        0       None   0
5    5  recommended_protocol     TEXT        0       None   0
                                  protocol_id protocol_type  stimulation_days  \
0       prot_antagonist_d9_dose3598.0_trg_hcg    Antagonist                 9   
1            prot_mild_d14_dose4051.0_trg_hcg          Mild                14   
2           prot_long_d14_dose4833.0_trg_dual          Long                14   
3       prot_antagonist_d9_dose3955.0_trg_hcg    Antagonist                 9   
4  prot_short_d11_dose3842.0_trg_gnrh_agonist         Short                11   

   total_fsh_dose  trigger_type recommended_protocol  
0          3598.0       

In [18]:
# Force close any active sqlite connection
import sqlite3
try:
    sqlite3.connect(STAR_DB).close()
except:
    pass

In [2]:
import sqlite3
import pandas as pd

# ====== PATH TO YOUR STAR DATABASE ======
db_path = r"E:\work\DEPI\graduation promax\data\warehouse\ivf_star_schema.db"

# ============ CONNECT =============
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ============ RUN QUERIES =============
queries = [
    "SELECT COUNT(*) FROM dim_protocol;",
    "SELECT COUNT(*) FROM dim_time;",
    "SELECT COUNT(*) FROM fact_transfer;",
    "SELECT * FROM dim_doctor LIMIT 5;",
    "SELECT * FROM fact_ivf_cycle LIMIT 5;"
]

for q in queries:
    print(f"\n--- Query: {q} ---")
    try:
        df = pd.read_sql(q, conn)   # 👈 gives neat table view
        print(df)
    except Exception as e:
        print("Error:", e)

# ============ CLOSE =============
conn.close()
print("\nConnection closed.")



--- Query: SELECT COUNT(*) FROM dim_protocol; ---
   COUNT(*)
0      9670

--- Query: SELECT COUNT(*) FROM dim_time; ---
   COUNT(*)
0      3376

--- Query: SELECT COUNT(*) FROM fact_transfer; ---
   COUNT(*)
0     10000

--- Query: SELECT * FROM dim_doctor LIMIT 5; ---
    doctor_id                              doctor_recommendation
0  dr_unknown  Increase gonadotropin dose in next cycle for b...
1  dr_unknown  Proceed with embryo freezing for future transfer.
2  dr_unknown  Monitor progesterone closely during luteal phase.
3  dr_unknown  Consider switching to mild stimulation protoco...
4  dr_unknown            Good prognosis, continue same protocol.

--- Query: SELECT * FROM fact_ivf_cycle LIMIT 5; ---
   cycle_sk     case_id                             female_id  \
0         1  CASE100000  427ba873-37d2-4dd4-9e2f-ee76fd8836fc   
1         2  CASE100001  279bd41e-7f8c-4531-8391-19cda6887977   
2         3  CASE100002  935afeb1-0f78-4af5-8ae7-a44627e5930e   
3         4  CASE100003 

In [38]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(r"E:\work\DEPI\graduation promax\data\raw\ivf_database_updated.db")
df = pd.read_sql("SELECT * FROM ivf_patients LIMIT 5;", conn)
print(df.columns)
conn.close()

Index(['case_id', 'et_date', 'female_age', 'female_bmi', 'amh_level',
       'fsh_level', 'afc', 'male_age', 'male_factor',
       'semen_count_mill_per_ml', 'motility_percent', 'morphology_percent',
       'protocol_type', 'stimulation_days', 'total_fsh_dose', 'trigger_type',
       'e2_on_trigger', 'endometrium_thickness', 'follicles_18mm',
       'retrieved_oocytes', 'mii_count', 'mi_count', 'gv_count',
       'injected_mii', 'fertilized_oocytes', 'fertilization_rate',
       'cleavage_d3', 'blastocyst_d5', 'good_embryos', 'class_a_rate',
       'fresh_et_stage', 'embryos_transferred', 'grading',
       'pregnancy_test_result', 'clinical_pregnancy', 'live_birth',
       'success_probability_score', 'response_type', 'risk_level',
       'recommended_protocol', 'suggested_waiting_period_days',
       'failure_reason', 'doctor_recommendation', 'female_id', 'male_id',
       'transfer_time_id'],
      dtype='object')


In [1]:
import sqlite3, pandas as pd
conn = sqlite3.connect(STAR_DB)

df = pd.read_sql("PRAGMA table_info(dim_time);", conn)
print(df)

conn.close()

NameError: name 'STAR_DB' is not defined